---
title: C++ socket 回声服务端的实现
tags: 小书匠,c++,socket,echo,networks
grammar_cjkRuby: true
renderNumberedHeading: true
---

[toc]

# C++ socket 回声服务端的实现

所谓回声服务器，顾名思义就是客户端发给它什么，它就返回给客户端什么。

## 代码

In [56]:
%%file server.cpp
#include <iostream>
#include <cstring> // for memset
#include <sys/socket.h>
#include <arpa/inet.h> // inet_addr
#include <netinet/in.h> // IPPROTO_TCP
#include <unistd.h>
#define BUFF_SIZE 100

int main() {
    int serv_sock = socket(AF_INET, SOCK_STREAM, IPPROTO_TCP);

    // 定义 serv 地址
    sockaddr_in serv_addr;
    memset(&serv_addr, 0, sizeof(serv_addr));
    serv_addr.sin_family = AF_INET; // 使用 IPv4
    serv_addr.sin_addr.s_addr = inet_addr("127.0.0.1"); //指定 ip
    serv_addr.sin_port = htons(1234); // 指定端口

    bind(serv_sock, (sockaddr*)&serv_addr, sizeof(serv_addr));

    listen(serv_sock, 20);
    
    // 定义 client 地址
    sockaddr_in client_addr;
    socklen_t client_addr_size = sizeof(client_addr);
    
    int client_sock = accept(serv_sock, (sockaddr*)&client_addr, &client_addr_size);
    printf("[Server] Client %s conect to the server!\n", inet_ntoa(client_addr.sin_addr));
    
    char buffer[BUFF_SIZE];
    
    int iret = 0;
    while (true)
    {
        iret = read(client_sock, buffer, BUFF_SIZE); // 接受数据
        if (iret > 0)
        {
            printf("[Server] Message from client: %s len: %d\n", buffer, iret);
            write(client_sock, buffer, sizeof(buffer)); // 原样返回
            printf("[Server] Message to client: %s\n", buffer);
        } else break;
    }
    close(client_sock);
    close(serv_sock);
    return 0;
}

Overwriting server.cpp


In [55]:
%%file client.cpp
#include <cstdio>
#include <unistd.h>
#include <arpa/inet.h>
#include <sys/socket.h>
#include <cstring>
#include <iostream>

int main()
{
    
    int sock = socket(AF_INET, SOCK_STREAM, IPPROTO_TCP);
    
    sockaddr_in serv_addr;
    memset(&serv_addr, 0, sizeof(serv_addr));
    serv_addr.sin_family = AF_INET;
    serv_addr.sin_addr.s_addr = inet_addr("127.0.0.1");
    serv_addr.sin_port = htons(1234);
    
    connect(sock, (sockaddr*)&serv_addr, sizeof(serv_addr)); // 连接
    
    // 获取用户输入
    char buffer[100];
    
    // 发送给服务器
    while (true)
    {
        std::cout << "[Client] Input a string:";
        std::cin >> buffer;
        write(sock, buffer, sizeof(buffer));
        printf("[Client] Message to server: %s\n", buffer);
        read(sock, buffer, sizeof(buffer)-1);
        printf("[Client] Message from server: %s\n", buffer);
        memset(buffer, 0, sizeof(buffer));
    }
    close(sock);
    return 0;
}

Overwriting client.cpp


In [46]:
!g++ server.cpp -o server
!g++ client.cpp -o client

# References
- [回声客户端的实现_C语言中文网](http://c.biancheng.net/cpp/html/3038.html)
- [实现迭代服务器端和客户端_C语言中文网](http://c.biancheng.net/cpp/html/3039.html)